# Müşteri yaşam Boyu Değeri

In [1]:
# 1. Veri Hazırlama
# 2. Average Order Value (average_order_value = total_price / total_transaction)
# 3. Purchase Frequency (total_transaction / total_number_of_customers)
# 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
# 5. Profit Margin (profit_margin =  total_price * 0.10)
# 6. Customer Value (customer_value = average_order_value * purchase_frequency)
# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
# 8. Segmentlerin Oluşturulması
# 9. BONUS: Tüm İşlemlerin Fonksiyonlaştırılması

# Customer Lifetime Value (CLTV), bir müşterinin bir şirketle kurduğu ilişki-iletişim süresince bu şirkete kazandırdığı parasal değerdir.
# CLTV hesaplanmasıyla, tahmini (prediction) farklı şeylerdir.

### 1. Veri Hazırlama

In [2]:
# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

# Değişkenler
#
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [4]:
# CLTV = (Customer Value / Churn Rate)*Profit Margin
# Customer Value = Average Order Value * Purchase Frequency
# Average Order Value = Total Price / Total Transaction         (DİKKAT! BURADAKİ TOTAL PRICE MÜŞTERİ ÖZELİNDEDİR !!!)
# Purchase Frequency = Total Transaction / Total Number of Customers
# Churn Rate = 1 - Repeat Rate
# Repeat Rate = Birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler
# Profit Margin = Total Price * kar oranı (Bu case'de 0.10 oalcaktır.)

# Churn Rate: Terk Oranı'dır. Customer Value'yi terk etme oranı ile düzeltiriz.


Her bir müşteri için hesaplanacak olan CLTV değerlerine göre sıralama yapıldığında ve CLTV değerlerine göre belirli noktalardan bölme işlemi yapılarak gruplar oluşturulduğunda müşteriler segmentlere ayrılacaktır.

In [5]:
yol = "datasets/online_retail_II.xlsx"

df_ = pd.read_excel(yol, sheet_name="Year 2009-2010")

df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [6]:
# Verideki boşluklara bakıyoruz;

df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [7]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [8]:
# Numerik değişkenlerin istatistiksel özet verilerine bakalım;

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.0,10.956631,104.354013,-9600.00,1.00,3.0,10.00,19152.00
Price,515255.0,3.956373,127.688563,-53594.36,1.25,2.1,4.21,25111.09
Customer ID,407695.0,15368.504107,1679.795700,12346.00,13997.00,15321.0,16812.00,18287.00


In [9]:
# Yukarıda baktığımızda quantity ve price'da negatif değerler var, miktar eksi olamayacağına göre bunları da biltereleyeceğiz,

df = df[df["Quantity"]>0]
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [10]:
# Betimsel istatistikleri bir kez daha kontrol ediyoruz;

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,513134.0,11.715433,92.974725,1.00,1.00,3.0,10.00,19152.00
Price,513134.0,3.972726,127.951933,-53594.36,1.25,2.1,4.21,25111.09
Customer ID,407695.0,15368.504107,1679.795700,12346.00,13997.00,15321.0,16812.00,18287.00


In [11]:
# Eksik değerleri siliyoruz;

df.dropna(inplace=True)
df

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28651/2796062761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [12]:
# Betimsel istatistikleri bir kez daha kontrol ediyoruz;

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407695.0,13.586686,96.842229,1.0,2.00,5.00,12.00,19152.0
Price,407695.0,3.294188,34.756655,0.0,1.25,1.95,3.75,10953.5
Customer ID,407695.0,15368.504107,1679.795700,12346.0,13997.00,15321.00,16812.00,18287.0


In [13]:
# Total Price Sütununu oluşturuyoruz.

df["TotalPrice"] = df["Quantity"]*df["Price"]
df

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_28651/2467803953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TotalPrice"] = df["Quantity"]*df["Price"]


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,7.50


In [14]:
cltv_c = df.groupby("Customer ID").agg({
                                            "Invoice"      : lambda x: x.nunique(),     
                                            # Her bir müşterinin fatura sayısına erişir ve total transaction'u görebiliriz.
                                            "Quantity"      : lambda x: x.sum(),         
                                            # Analiz etmek ve gözlemlemek için yazıyoruz.
                                            "TotalPrice"   : lambda x: x.sum()         
                                            # Müşteri başı total satın alımı hesaplar.
})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']
cltv_c

# Bakıldığında ilk sıradaki 12346 no'lu müşteri için 11 adet fatura kesilmiş, 
# 70 adet ürün almış ve aldığı ürünlerin total miktarı 372.68 TL veya USD tutmuş.

,total_transaction,total_unit,total_price
Customer ID,,,
12346.0,11,70,372.86
12347.0,2,828,1323.32
12348.0,1,373,222.16
12349.0,3,993,2671.14
12351.0,1,261,300.93
...,...,...,...
18283.0,6,336,641.77
18284.0,1,494,461.68
18285.0,1,145,427.00


### 2. Average Order Value (average_order_value = total_price / total_transaction)

In [15]:
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.0,11,70,372.86
12347.0,2,828,1323.32
12348.0,1,373,222.16
12349.0,3,993,2671.14
12351.0,1,261,300.93


In [16]:
# Hesaplanması;

cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.0,11,70,372.86,33.896364
12347.0,2,828,1323.32,661.660000
12348.0,1,373,222.16,222.160000
12349.0,3,993,2671.14,890.380000
12351.0,1,261,300.93,300.930000


### 3. Purchase Frequency (total_transaction / total_number_of_customers)

In [17]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.0,11,70,372.86,33.896364
12347.0,2,828,1323.32,661.660000
12348.0,1,373,222.16,222.160000
12349.0,3,993,2671.14,890.380000
12351.0,1,261,300.93,300.930000


In [18]:
cltv_c.shape

# Bakılıdığında 4314 adet müşteri var.

(4314, 4)

In [19]:
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

# Yukarıdaki tuple'ın ilk elemanına yani müşteri sayısına böldük.
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.0,11,70,372.86,33.896364,0.002550
12347.0,2,828,1323.32,661.660000,0.000464
12348.0,1,373,222.16,222.160000,0.000232
12349.0,3,993,2671.14,890.380000,0.000695
12351.0,1,261,300.93,300.930000,0.000232
...,...,...,...,...,...
18283.0,6,336,641.77,106.961667,0.001391
18284.0,1,494,461.68,461.680000,0.000232
18285.0,1,145,427.00,427.000000,0.000232


### 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)

In [20]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]
repeat_rate

0.6706073249884098

In [21]:
churn_rate = 1 - repeat_rate
churn_rate

0.3293926750115902

### 5. Profit Margin (profit_margin =  total_price * 0.10)

In [22]:
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.0,11,70,372.86,33.896364,0.002550,37.286
12347.0,2,828,1323.32,661.660000,0.000464,132.332
12348.0,1,373,222.16,222.160000,0.000232,22.216
12349.0,3,993,2671.14,890.380000,0.000695,267.114
12351.0,1,261,300.93,300.930000,0.000232,30.093
...,...,...,...,...,...,...
18283.0,6,336,641.77,106.961667,0.001391,64.177
18284.0,1,494,461.68,461.680000,0.000232,46.168
18285.0,1,145,427.00,427.000000,0.000232,42.700


### 6. Customer Value (customer_value = average_order_value * purchase_frequency)

In [23]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]
cltv_c

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.0,11,70,372.86,33.896364,0.002550,37.286,0.086430
12347.0,2,828,1323.32,661.660000,0.000464,132.332,0.306750
12348.0,1,373,222.16,222.160000,0.000232,22.216,0.051497
12349.0,3,993,2671.14,890.380000,0.000695,267.114,0.619179
12351.0,1,261,300.93,300.930000,0.000232,30.093,0.069757
...,...,...,...,...,...,...,...
18283.0,6,336,641.77,106.961667,0.001391,64.177,0.148764
18284.0,1,494,461.68,461.680000,0.000232,46.168,0.107019
18285.0,1,145,427.00,427.000000,0.000232,42.700,0.098980


### 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [24]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.0,89,124216,349164.35,3923.194944,0.020631,34916.435,80.937494,8.579574e+06
14646.0,78,170342,248396.50,3184.570513,0.018081,24839.650,57.579161,4.342070e+06
14156.0,102,108107,196566.74,1927.124902,0.023644,19656.674,45.564845,2.719105e+06
14911.0,205,69722,152147.57,742.183268,0.047520,15214.757,35.268329,1.629056e+06
13694.0,94,125893,131443.19,1398.331809,0.021790,13144.319,30.468982,1.215856e+06


### 8. Segmentlerin Oluşturulması

In [25]:
# Müşteri sayısının fazla olmasından ve farklı önem düzeylerinde olmalarından kaynaklı olarak 
# müşterilerimizi segmentlere ayırmak isteriz.

# Müşterilerimizi 4 gruba ayırarak, grup içinde analiz yapabiliriz.

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])
# cltv'sini küçük gördüğüne "D" büyük gördüğüne ise "A" verecek şekilde ayarladık.

cltv_c


,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.0,11,70,372.86,33.896364,0.002550,37.286,0.086430,9.783574,C
12347.0,2,828,1323.32,661.660000,0.000464,132.332,0.306750,123.235455,B
12348.0,1,373,222.16,222.160000,0.000232,22.216,0.051497,3.473263,D
12349.0,3,993,2671.14,890.380000,0.000695,267.114,0.619179,502.110408,A
12351.0,1,261,300.93,300.930000,0.000232,30.093,0.069757,6.372897,D
...,...,...,...,...,...,...,...,...,...
18283.0,6,336,641.77,106.961667,0.001391,64.177,0.148764,28.984429,C
18284.0,1,494,461.68,461.680000,0.000232,46.168,0.107019,14.999889,C
18285.0,1,145,427.00,427.000000,0.000232,42.700,0.098980,12.831034,C


In [26]:
pd.set_option('display.max_columns', None)

# Yukarıda bütün sütunları görebilmek adına bir ayar yaptık.

In [27]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

# Aşağıdaki tabloya baktığımızda;
# A segmentinin toplam işlem ortalamasının 10.8, toplam işlem sayısının ise 11666 olduğunu görüyoruz.
# 4'e böldüğümüz için her bir segmentteki müşteri sayısının eşit olmasını zaten bekleriz.
# B segmentinin toplam alışveriş miktarının ortalamasının  1.1 bin, toplam alışverişinin ise 1.2 milyon olduğunu görüyoruz.

total_transaction                  total_unit                     \
                      sum       mean count        sum         mean count   
segment                                                                    
D                    1326   1.228916  1079     117616   109.004634  1079   
C                    2160   2.003711  1078     305135   283.056586  1078   
B                    4063   3.769017  1078     733211   680.158627  1078   
A                   11666  10.811863  1079    4383262  4062.337349  1079   

         total_price                    average_order_value                    \
                 sum         mean count                 sum        mean count   
segment                                                                         
D         192265.130   178.188258  1079       169805.465496  157.372999  1079   
C         513016.453   475.896524  1078       317422.355141  294.454875  1078   
B        1219605.200  1131.359184  1078       420127.123652  389.728315  1078   
A        6907116.491  6401.405460  1079       723972.223469  670.965916  1079   

        purchase_frequency                 profit_margin                    \
                       sum      mean count           sum        mean count   
segment                                                                      
D                 0.307371  0.000285  1079    19226.5130   17.818826  1079   
C                 0.500695  0.000464  1078    51301.6453   47.589652  1078   
B                 0.941817  0.000874  1078   121960.5200  113.135918  1078   
A                 2.704219  0.002506  1079   690711.6491  640.140546  1079   

        customer_value                          cltv                      
                   sum      mean count           sum          mean count  
segment                                                                   
D            44.567717  0.041305  1079  2.849328e+03      2.640712  1079  
C           118.918974  0.110314  1078  1.818426e+04     16.868519  1078  
B           282.708669  0.262253  1078  1.035499e+05     96.057387  1078  
A          1601.093299  1.483868  1079  2.525730e+07  23408.058426  1079